In [10]:
import numpy as np
import pandas as pd
import pickle

In [11]:
labeled_tags = pd.read_csv("Labeled.csv")
file = open("temp_model.pickle",'rb')
df = pickle.load(file)
file.close()
lats = []
lons = []

for row in df['RLocation']:
    if type(row) != str:
        lat,lon = (None,None)
    else:
        lat,lon = tuple(str(row).split(','))
        lat = int(lat)
        lon = int(lon)
    lats.append(lat)
    lons.append(lon)
df["Lat"] = lats
df["Long"] = lons 

df_located = df[['RLocation','Lat','Long']][df['RLocation'].notna()]
X = np.array(df_located)
X = X[:,1:3]

In [12]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

In [13]:
X = StandardScaler().fit_transform(X)
db = DBSCAN(eps=0.1, min_samples=2).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

df_located["unit_label"] = db.labels_
df["unit_label"] = None
df.iloc[df_located.index, -1] = db.labels_

groupby_unit_label_Lat = df[df['Lat'].notnull()].groupby('unit_label').mean()[['Lat','Long']]
groupby_unit_label_Lat = groupby_unit_label_Lat.rename(columns={'Lat':'Lat_mean','Long':'Long_mean'})
df = df.merge(groupby_unit_label_Lat, how = 'left', right_index = True, left_on = 'unit_label')
df[['Area No','Unit No','Sequence Number']] = df[['Area No','Unit No','Sequence Number']].applymap(str)
df_encoded = pd.get_dummies(df[['DWG Number', 'Area No','Unit No']], prefix=['DWG Number', 'Area No','Unit No'])
normalized_df = df_encoded
train_index = normalized_df[df["RLocation"].notna()].dropna().index

X = np.array(normalized_df[df["RLocation"].notna()].dropna())
y = np.array(pd.get_dummies(df.iloc[train_index,:]["unit_label"]))

from sklearn.neighbors import KNeighborsClassifier
model_knn= KNeighborsClassifier(n_neighbors=3, metric='jaccard')
model_knn.fit(X,y)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='jaccard',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [14]:
test_index = normalized_df[df["RLocation"].isna()].dropna().index
X_test = np.array(normalized_df.iloc[test_index,:])
y_pred = model_knn.predict(X_test)

y_pred_arr = []
No_Out_list = []
for j, row in enumerate(y_pred):
    for i,v in enumerate(row):
        if v: y_pred_arr = np.append(y_pred_arr,i)
    if sum(row) == 0: 
        print("nothing at j: " + str(j))
        No_Out_list = np.append(No_Out_list,j)
        y_pred_arr = np.append(y_pred_arr, None)

df['pre_unit_label'] = None
df['pre_unit_label'] = pd.Series(y_pred_arr,index=test_index)
groupby_unit_label_Lat = groupby_unit_label_Lat.rename(columns={'Lat_mean':'pre_Lat_mean','Long_mean':'pre_Long_mean'})
df = df.merge(groupby_unit_label_Lat, how = 'left', right_index = True, left_on = 'pre_unit_label')
check_result_df = df[['DWG Number', 'Tag', 'Signal Type 1', 'Lat',
       'Long', 'unit_label', 'Lat_mean', 'Long_mean', 'pre_unit_label', 'pre_Lat_mean', 'pre_Long_mean']]
combined_location_df = check_result_df[['Lat_mean', 'Long_mean',
       'pre_Lat_mean', 'pre_Long_mean']].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)
combined_location_df = combined_location_df.str.split(',',expand=True)
check_result_df[['cL_Lat', 'cL_Long']] = combined_location_df
plot_df = check_result_df[['Tag','cL_Lat','cL_Long']]
plot_df = plot_df[['cL_Lat', 'cL_Long']].apply(pd.to_numeric)
plot_bygroup_df = plot_df.groupby(['cL_Lat', 'cL_Long']).size().reset_index(name='count')
nonNAN_index = plot_df[['cL_Lat', 'cL_Long']].dropna().index
X = np.array(plot_df.iloc[nonNAN_index,:][['cL_Lat', 'cL_Long']] )


C:\Users\Zhimin90\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [15]:
from dbscan import MyDBSCAN
from sklearn.preprocessing import StandardScaler

In [16]:
X_normalized = StandardScaler().fit_transform(X)
my_labels = np.array(MyDBSCAN(X_normalized, eps=.2, MinPts=20, MaxPts = 100))
core_samples_mask = np.zeros_like(my_labels, dtype=bool)
core_samples_mask[np.array(range(0,len(my_labels)))] = True
labels = my_labels
clusterDict = {}
for label in labels:
    if label in clusterDict.keys():
        clusterDict[label] += 1
    else:
        clusterDict[label] = 1
dbscan_labeled = check_result_df.iloc[nonNAN_index,:]
dbscan_labeled["Gateway_Label"] = my_labels


{}
Growing Cluster
Completed
{1: 96}
Growing Cluster
Completed
{1: 96, 2: 18}
Growing Cluster
Completed
{1: 96, 2: 35}
Growing Cluster
Completed
{1: 96, 2: 52}
Growing Cluster
Completed
{1: 96, 2: 74}
Growing Cluster
{1: 96, 2: 101}
Growing Cluster
Completed
{1: 96, 2: 101, 3: 65}
Growing Cluster
{1: 96, 2: 101, 3: 101}
Growing Cluster
Completed


In [17]:
export_df = df[['DWG Number','Tag','Description','Signal Type 1']]
export_df['ASIGateway_label'] = dbscan_labeled['Gateway_Label']

C:\Users\Zhimin90\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
export_df

,DWG Number,Tag,Description,Signal Type 1,ASIGateway_label
0,PID-143,20-ZSC-7753.,Closed Detection,DIASI,1
1,PID-143,20-XY-4101C,3-Way Solenoid Valve,DOASI,1
2,PID-143,20-ZSC-4101A,Closed Detection,DIASI,1
3,PID-143,20-ZIS-4102K,Pig Sensor,DIASI,1
4,PID-143,20-XY-4101E,3-Way Solenoid Valve,DOASI,1
...,...,...,...,...,...
347,PID-142,20-ZSC-4103G,Closed Detection,DIASI,1
348,PID-142,20-ZIS-4105K,Pig Sensor,DIASI,1
349,PID-142,20-ZSC/O-4104E,Closed and Open Detection,DIASI,1
350,PID-142,20-ZIS-4103J,Pig Sensor,DIASI,1


In [19]:
file = open("temp_model.pickle",'rb')
df = pickle.load(file)
df['ASIGateway_label'] = dbscan_labeled['Gateway_Label']

with open('learnedmodel.pickle', 'wb') as f:
    pickle.dump(df, f)